In [7]:
import numpy as np
import librosa
import cv2
import moviepy.editor as mpy

class GeometricShape:
    """
    幾何学図形（長方形または線）を表すクラス。

    Attributes:
        shape_type (str): 図形の種類 ("rectangle" または "line")。
        width (int): 図形を描画するフレームの幅。
        height (int): 図形を描画するフレームの高さ。
        color (tuple): 図形の色 (R, G, B) のタプル。デフォルトは白 (255, 255, 255)。
        opacity (float): 図形の不透明度。0 は完全に透明、1 は完全に不透明。デフォルトは 1。
        size (tuple): 長方形の幅と高さのタプル。
        position (tuple): 図形の左上隅の位置 (x, y) のタプル。
        growth_rate (float): 長方形の拡大率。
        thickness (int): 線の太さ。
        height_percentage (float): 線の高さを表す割合。0 は高さ 0、1 は最大の高さ。
    """
    def __init__(self, shape_type, width, height):
        """
        GeometricShape オブジェクトを初期化する。

        Args:
            shape_type (str): 図形の種類 ("rectangle" または "line")。
            width (int): 図形を描画するフレームの幅。
            height (int): 図形を描画するフレームの高さ。
        """
        self.shape_type = shape_type
        self.width = width
        self.height = height
        self.color = (255, 255, 255)
        self.opacity = 1
        self.reset()

    def reset(self):
        """
        図形のサイズ、位置、成長率をランダムにリセットする。
        """
        if self.shape_type == 'rectangle':
            self.size = (np.random.randint(50, 100), np.random.randint(50, 100))
            self.position = (np.random.randint(0, self.width - self.size[0]),
                             np.random.randint(0, self.height - self.size[1]))
            self.growth_rate = np.random.uniform(1.01, 1.05)
        elif self.shape_type == 'line':
            self.thickness = np.random.randint(2, 5)
            self.position = np.random.randint(0, self.width)
            self.height_percentage = 0

    def update(self, energy, current_onset, max_onset):
        """
        オーディオのエネルギーとオンセット強度に基づいて図形を更新する。

        Args:
            energy (float): オーディオのエネルギー。
            current_onset (float): 現在のオンセット強度。
            max_onset (float): 最大オンセット強度。
        """
        if self.shape_type == 'rectangle':
            self.size = (int(self.size[0] * self.growth_rate),
                         int(self.size[1] * self.growth_rate))
            if self.size[0] > self.width // 2 or self.size[1] > self.height // 2:
                self.reset()
        elif self.shape_type == 'line':
            self.height_percentage = min(1, self.height_percentage + energy * 0.1)

    def draw(self, frame, bounce_offset):
        """
        指定されたフレームに図形を描画する。

        Args:
            frame (numpy.ndarray): 図形を描画するフレーム。
            bounce_offset (int): バウンス効果のオフセット。

        Returns:
            numpy.ndarray: 図形が描画されたフレーム。
        """
        if self.shape_type == 'rectangle':
            cv2.rectangle(frame,
                          (self.position[0], self.position[1] + bounce_offset),
                          (self.position[0] + self.size[0], self.position[1] + self.size[1] + bounce_offset),
                          self.color, 2)
        elif self.shape_type == 'line':
            start_point = (self.position, self.height + bounce_offset)
            end_point = (self.position, int(self.height * (1 - self.height_percentage)) + bounce_offset)
            cv2.line(frame, start_point, end_point, self.color, self.thickness)
        return frame

def create_shapes(width, height):
    """
    ランダムな形状のリストを作成する。

    Args:
        width (int): 図形を描画するフレームの幅。
        height (int): 図形を描画するフレームの高さ。

    Returns:
        list: GeometricShape オブジェクトのリスト。
    """
    shapes = [GeometricShape('rectangle', width, height) for _ in range(5)]
    line_positions = np.linspace(width // 4, 3 * width // 4, 3, dtype=int)
    shapes.extend([GeometricShape('line', width, height) for _ in line_positions])
    return shapes

def create_particles(num_particles, width, height):
    """
    指定された数のパーティクルを作成する。

    Args:
        num_particles (int): 作成するパーティクルの数。
        width (int): パーティクルを描画するフレームの幅。
        height (int): パーティクルを描画するフレームの高さ。

    Returns:
        numpy.ndarray: パーティクルの位置 (x, y)、サイズ、速度を表す配列。
    """
    particles = np.random.rand(num_particles, 4)
    particles[:, 0] *= width
    particles[:, 1] *= height
    particles[:, 2] = np.random.randint(2, 5, num_particles)
    particles[:, 3] = np.random.randint(1, 5, num_particles)
    return particles


def create_frosted_glass_blocks(width, height, block_size_range=(50, 150), num_blocks=10):
    """
    フロストガラス効果を適用するためのブロックのリストを作成する。

    Args:
        width (int): フレームの幅。
        height (int): フレームの高さ。
        block_size_range (tuple): ブロックのサイズ範囲 (最小、最大)。
        num_blocks (int): 作成するブロックの数。

    Returns:
        list: 各ブロックの (x, y, 幅, 高さ) を表すタプルのリスト。
    """
    blocks = []
    for _ in range(num_blocks):
        block_width = np.random.randint(block_size_range[0], block_size_range[1])
        block_height = np.random.randint(block_size_range[0], block_size_range[1])
        block_x = np.random.randint(0, width - block_width)
        block_y = np.random.randint(0, height - block_height)
        blocks.append((block_x, block_y, block_width, block_height))
    return blocks

def apply_frosted_glass_effect(frame, blocks, blur_strength=15):
    """
    フレームにフロストガラス効果を適用する。

    Args:
        frame (numpy.ndarray): フロストガラス効果を適用するフレーム。
        blocks (list): 各ブロックの (x, y, 幅, 高さ) を表すタプルのリスト。
        blur_strength (int): ぼかしの強さ。

    Returns:
        numpy.ndarray: フロストガラス効果が適用されたフレーム。
    """
    result = frame.copy()
    for x, y, w, h in blocks:
        roi = result[y:y+h, x:x+w]
        
        # ガウシアンブラーを適用
        blurred = cv2.GaussianBlur(roi, (blur_strength, blur_strength), 0)
        
        # 透明度マップを作成
        alpha = np.random.uniform(0.3, 0.7, size=(h, w, 1)).astype(np.float32)
        
        # ぼかした領域と元の領域をブレンド
        frosted = cv2.addWeighted(blurred, alpha[..., 0], roi, 1 - alpha[..., 0], 0)
        
        # エッジを強調
        edges = cv2.Canny(frosted, 100, 200)
        frosted = cv2.addWeighted(frosted, 0.9, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.1, 0)
        
        # 明るさとコントラストを調整
        frosted = cv2.addWeighted(frosted, 1.1, np.zeros_like(frosted), 0, 5)
        
        # フロスト効果を領域に適用
        result[y:y+h, x:x+w] = frosted
    
    return result

def update_particles(particles, width, height, energy):
    """
    オーディオのエネルギーに基づいてパーティクルを更新する。

    Args:
        particles (numpy.ndarray): パーティクルの位置 (x, y)、サイズ、速度を表す配列。
        width (int): パーティクルを描画するフレームの幅。
        height (int): パーティクルを描画するフレームの高さ。
        energy (float): オーディオのエネルギー。

    Returns:
        numpy.ndarray: 更新されたパーティクルの位置 (x, y)、サイズ、速度を表す配列。
    """
    particles[:, 1] -= particles[:, 3] * energy
    reset = particles[:, 1] < 0
    particles[reset, 1] = height
    particles[reset, 0] = np.random.randint(0, width, np.sum(reset))
    return particles

def draw_particles(frame, particles, color):
    """
    指定されたフレームにパーティクルを描画する。

    Args:
        frame (numpy.ndarray): パーティクルを描画するフレーム。
        particles (numpy.ndarray): パーティクルの位置 (x, y)、サイズ、速度を表す配列。
        color (tuple): パーティクルの色 (R, G, B) のタプル。
    """
    for x, y, size, _ in particles:
        cv2.circle(frame, (int(x), int(y)), int(size), color, -1)

def apply_blur_effect(frame, energy):
    """
    オーディオのエネルギーに基づいてフレームにぼかし効果を適用する。

    Args:
        frame (numpy.ndarray): ぼかし効果を適用するフレーム。
        energy (float): オーディオのエネルギー。

    Returns:
        numpy.ndarray: ぼかし効果が適用されたフレーム。
    """
    blur_amount = max(1, int(energy * 2))
    return cv2.GaussianBlur(frame, (blur_amount * 2 + 1, blur_amount * 2 + 1), 0)

def apply_bounce_effect(frame, current_onset, max_onset, original_height):
    """
    オーディオのオンセット強度に基づいてフレームにバウンス効果を適用する。

    Args:
        frame (numpy.ndarray): バウンス効果を適用するフレーム。
        current_onset (float): 現在のオンセット強度。
        max_onset (float): 最大オンセット強度。
        original_height (int): 元のフレームの高さ。

    Returns:
        tuple: バウンス効果が適用されたフレームとバウンスのオフセット。
    """
    bounce_amount = int(30 * current_onset / max_onset)
    if bounce_amount > 0:
        frame = frame[bounce_amount:-bounce_amount, :]
    return cv2.resize(frame, (frame.shape[1], original_height)), bounce_amount

def apply_frosted_glass_effect(frame, blocks, blur_strength=15):
    """
    フレームにすりガラス効果を適用する。

    Args:
        frame (numpy.ndarray): すりガラス効果を適用するフレーム。
        blocks (list): 各ブロックの (x, y, 幅, 高さ) を表すタプルのリスト。
        blur_strength (int): ぼかしの強さ。

    Returns:
        numpy.ndarray: すりガラス効果が適用されたフレーム。
    """
    result = frame.copy()
    for x, y, w, h in blocks:
        roi = result[y:y+h, x:x+w]
        
        # ガウシアンブラーを適用
        blurred = cv2.GaussianBlur(roi, (blur_strength, blur_strength), 0)
        
        # コントラストを増加
        mean = cv2.mean(blurred)[0]
        contrast_enhanced = cv2.addWeighted(blurred, 1.5, blurred, 0, mean * -0.5)
        
        # エッジを強調
        edge_enhanced = cv2.addWeighted(contrast_enhanced, 1.5, cv2.Laplacian(contrast_enhanced, cv2.CV_8U), 0.5, 0)
        
        # 元の画像とブレンド
        alpha = 0.7
        result[y:y+h, x:x+w] = cv2.addWeighted(roi, alpha, edge_enhanced, 1-alpha, 0)
    
    return result


def apply_glitch_effect(frame, strength=10):
    """
    フレームにグリッチ効果を適用する。

    Args:
        frame (numpy.ndarray): グリッチ効果を適用するフレーム。
        strength (int): グリッチの強さ。

    Returns:
        numpy.ndarray: グリッチ効果が適用されたフレーム。
    """
    height, width, _ = frame.shape
    glitch_frame = frame.copy()
    num_slices = np.random.randint(1, strength)
    for _ in range(num_slices):
        slice_height = np.random.randint(1, height // strength)
        start_y = np.random.randint(0, height - slice_height)
        start_x = np.random.randint(-strength, strength)
        end_x = width + start_x
        if start_x > 0:
            end_x = min(end_x, width)
            glitch_frame[start_y:start_y + slice_height, start_x:end_x] = frame[start_y:start_y + slice_height, :end_x - start_x]
        else:
            start_x = max(start_x, -width)
            glitch_frame[start_y:start_y + slice_height, :end_x] = frame[start_y:start_y + slice_height, -start_x:]
    return glitch_frame

def apply_trailing_effect(frame, previous_frames):
    """
    前のフレームの軌跡効果を適用する。

    Args:
        frame (numpy.ndarray): 軌跡効果を適用するフレーム。
        previous_frames (list): 前のフレームのリスト。

    Returns:
        numpy.ndarray: 軌跡効果が適用されたフレーム。
    """
    alpha = 0.7
    result = frame.copy()
    for prev_frame in previous_frames:
        result = cv2.addWeighted(result, alpha, prev_frame, 1 - alpha, 0)
    return result

def apply_color_shift(frame, shift_amount):
    """
    フレームに色のずれ効果を適用する。

    Args:
        frame (numpy.ndarray): 色のずれ効果を適用するフレーム。
        shift_amount (int): ずらす量。

    Returns:
        numpy.ndarray: 色のずれ効果が適用されたフレーム。
    """
    b, g, r = cv2.split(frame)
    b = np.roll(b, shift_amount, axis=1)
    r = np.roll(r, -shift_amount, axis=1)
    return cv2.merge([b, g, r])

def create_music_visualizer(image_path, audio_path, output_path):
    """
    音楽ビジュアライザーを作成する。

    Args:
        image_path (str): 入力画像ファイルのパス。
        audio_path (str): 入力オーディオファイルのパス。
        output_path (str): 出力ビデオファイルのパス。
    """
    y, sr = librosa.load(audio_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    duration = librosa.get_duration(y=y, sr=sr)
    fps = 30
    n_frames = int(duration * fps)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=32, fmax=8000, hop_length=sr // fps)
    S_dB = librosa.power_to_db(S, ref=np.max)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=sr // fps)

    height, width, _ = img.shape
    particles = create_particles(100, width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('temp_output.mp4', fourcc, fps, (width, height))

    prev_bars = np.zeros(32)
    zoom_factor = 1.0
    max_onset = np.max(onset_env)
    previous_frames = []

    shapes = create_shapes(width, height)

    # フロストガラスブロックを作成
    frosted_blocks = create_frosted_glass_blocks(width, height, block_size_range=(50, 150), num_blocks=15)

    for frame_num in range(n_frames):
        audio_idx = int(frame_num * len(y) / n_frames)
        chunk = y[audio_idx:audio_idx + sr // fps]

        spec_frame = S_dB[:, min(frame_num, S_dB.shape[1] - 1)]
        current_onset = onset_env[min(frame_num, len(onset_env) - 1)]
        energy = np.mean(np.abs(chunk)) * 10

        frame = img.copy()
        particles = update_particles(particles, width, height, energy)
        draw_particles(frame, particles, (255, 255, 255))

        bar_width = width // 32
        max_bar_height = height // 4

        target_heights = np.interp(spec_frame, [S_dB.min(), S_dB.max()], [0, max_bar_height])
        prev_bars = prev_bars * 0.7 + target_heights * 0.3
        bar_heights = prev_bars.astype(int)

        overlay = frame.copy()

        for j, bar_height in enumerate(bar_heights):
            cv2.rectangle(overlay,
                          (j * bar_width, height // 2 - bar_height),
                          ((j + 1) * bar_width, height // 2),
                          (255, 255, 255, 150),
                          -1)
            cv2.rectangle(overlay,
                          (j * bar_width, height // 2),
                          ((j + 1) * bar_width, height // 2 + bar_height),
                          (255, 255, 255, 150),
                          -1)

        alpha = 0.6
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        frame = apply_blur_effect(frame, energy)
        target_zoom = 1 + 0.05 * current_onset / max_onset
        zoom_factor = zoom_factor * 0.7 + target_zoom * 0.3
        scaled_frame = cv2.resize(frame, None, fx=zoom_factor, fy=zoom_factor)
        start_y = (scaled_frame.shape[0] - height) // 2
        start_x = (scaled_frame.shape[1] - width) // 2
        frame = scaled_frame[start_y:start_y + height, start_x:start_x + width]
        frame, bounce_offset = apply_bounce_effect(frame, current_onset, max_onset, height)

        frame = apply_color_shift(frame, int(energy * 10))

        if np.random.rand() < 0.1:
            frame = apply_glitch_effect(frame)

        if len(previous_frames) > 5:
            previous_frames.pop(0)
        previous_frames.append(frame.copy())
        frame = apply_trailing_effect(frame, previous_frames)

        for shape in shapes:
            shape.update(energy, current_onset, max_onset)
            frame = shape.draw(frame, bounce_offset)

       
        frame = apply_frosted_glass_effect(frame, frosted_blocks, blur_strength=45)

        video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    video.release()

    video = mpy.VideoFileClip('temp_output.mp4')
    audio = mpy.AudioFileClip(audio_path).set_duration(video.duration)
    final_video = video.set_audio(audio)
    final_video.write_videofile(output_path, codec="libx264", audio_codec="aac")
# 画像ファイルとオーディオファイルへのパス
image_path = '030b542c0f69faf2c54a49a594267f25_waifu2x_2x_jpg_waifu2x_2x_png.png'
audio_path = '「神戸Psynthwave夜景」.mp3'  
output_path = 'output_video.mp4'
create_music_visualizer(image_path, audio_path, output_path)


error: OpenCV(4.10.0) D:\bld\libopencv_1718893052016\work\modules\imgproc\src\smooth.dispatch.cpp:294: error: (-215:Assertion failed) ksize.width > 0 && ksize.width % 2 == 1 && ksize.height > 0 && ksize.height % 2 == 1 in function 'cv::createGaussianKernels'
